In [1]:
import os
import cv2
import numpy as np
from google.colab import drive
import albumentations as A

# Mount Google Drive
drive.mount('/content/drive')

# Path dataset
path = "/content/drive/MyDrive/pmk_dataset/yolo/gusi/train"

Mounted at /content/drive


In [2]:
# Target jumlah foto per kelas
TARGET_PHOTOS = 30

# Definisi augmentasi (ringan untuk data medis)
def get_augmentation():
    return A.Compose([
        A.OneOf([
            A.Rotate(limit=15, p=0.8),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.GaussianBlur(blur_limit=3, p=0.3),
        ], p=1.0)
    ])

In [3]:
# Fungsi untuk augmentasi per kelas
def augment_class(class_folder_path, class_name, target_count):
    print(f"\n=== Memproses Kelas {class_name} ===")

    # Dapatkan semua file gambar
    image_files = [f for f in os.listdir(class_folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    original_count = len(image_files)
    needed_count = target_count - original_count

    print(f"Foto asli: {original_count}")
    print(f"Foto dibutuhkan: {needed_count}")

    if needed_count <= 0:
        print("Sudah cukup, tidak perlu augmentasi!")
        return

    # Siapkan augmentasi
    transform = get_augmentation()

    # Hitung berapa kali harus mengulang
    augmentations_per_image = needed_count // original_count
    extra_augmentations = needed_count % original_count

    print(f"Augmentasi per foto: {augmentations_per_image}")
    print(f"Augmentasi tambahan: {extra_augmentations}")

    counter = 1

    # Lakukan augmentasi
    for i, image_file in enumerate(image_files):
        # Baca gambar
        image_path = os.path.join(class_folder_path, image_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Nama dasar untuk file augmented
        base_name = os.path.splitext(image_file)[0]

        # Augmentasi reguler
        for j in range(augmentations_per_image):
            augmented = transform(image=image)['image']
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            # Simpan dengan nama yang dinomori
            new_name = f"{base_name}_{counter:02d}.jpg"
            save_path = os.path.join(class_folder_path, new_name)
            cv2.imwrite(save_path, augmented_bgr)

            counter += 1
            print(f"  Dibuat: {new_name}")

        # Augmentasi tambahan (jika diperlukan)
        if i < extra_augmentations:
            augmented = transform(image=image)['image']
            augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

            new_name = f"{base_name}_{counter:02d}.jpg"
            save_path = os.path.join(class_folder_path, new_name)
            cv2.imwrite(save_path, augmented_bgr)

            counter += 1
            print(f"  Dibuat: {new_name}")

    print(f"Selesai! Total foto sekarang: {len(os.listdir(class_folder_path))}")

In [4]:
# Jalankan augmentasi untuk semua kelas
def main():
    # Install albumentations jika belum ada
    try:
        import albumentations
    except ImportError:
        print("Installing albumentations...")
        os.system("pip install albumentations")
        import albumentations as A

    print("=== MULAI AUGMENTASI DATA PMK ===")
    print(f"Target: {TARGET_PHOTOS} foto per kelas")

    # Loop untuk setiap kelas (0, 1, 2, 3)
    classes = ['0', '1', '2', '3']

    for class_name in classes:
        class_path = os.path.join(path, class_name)

        if os.path.exists(class_path):
            augment_class(class_path, class_name, TARGET_PHOTOS)
        else:
            print(f"Folder kelas {class_name} tidak ditemukan!")

    print("\n=== AUGMENTASI SELESAI ===")

    # Tampilkan summary
    print("\n=== SUMMARY ===")
    for class_name in classes:
        class_path = os.path.join(path, class_name)
        if os.path.exists(class_path):
            total_files = len([f for f in os.listdir(class_path)
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            print(f"Kelas {class_name}: {total_files} foto")

# Jalankan program
if __name__ == "__main__":
    main()

=== MULAI AUGMENTASI DATA PMK ===
Target: 30 foto per kelas

=== Memproses Kelas 0 ===
Foto asli: 9
Foto dibutuhkan: 21
Augmentasi per foto: 2
Augmentasi tambahan: 3


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: IMG_20221105_104658_01.jpg
  Dibuat: IMG_20221105_104658_02.jpg
  Dibuat: IMG_20221105_104658_03.jpg
  Dibuat: IMG_20221105_104657_04.jpg
  Dibuat: IMG_20221105_104657_05.jpg
  Dibuat: IMG_20221105_104657_06.jpg
  Dibuat: IMG_20221105_104656_07.jpg
  Dibuat: IMG_20221105_104656_08.jpg
  Dibuat: IMG_20221105_104656_09.jpg
  Dibuat: IMG_20221105_104654_10.jpg
  Dibuat: IMG_20221105_104654_11.jpg
  Dibuat: IMG_20221105_104824_12.jpg
  Dibuat: IMG_20221105_104824_13.jpg
  Dibuat: IMG_20221105_104823_14.jpg
  Dibuat: IMG_20221105_104823_15.jpg
  Dibuat: IMG_20221105_104655_16.jpg
  Dibuat: IMG_20221105_104655_17.jpg
  Dibuat: IMG_20221105_104700_18.jpg
  Dibuat: IMG_20221105_104700_19.jpg
  Dibuat: IMG_20221105_104653_20.jpg
  Dibuat: IMG_20221105_104653_21.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 1 ===
Foto asli: 6
Foto dibutuhkan: 24
Augmentasi per foto: 4
Augmentasi tambahan: 0


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: IMG_0772-1000x600_01.jpg
  Dibuat: IMG_0772-1000x600_02.jpg
  Dibuat: IMG_0772-1000x600_03.jpg
  Dibuat: IMG_0772-1000x600_04.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_05.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_06.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_07.jpg
  Dibuat: ed0fc765a034835e4c0eb5d4cd4dfbca_08.jpg
  Dibuat: 48597919241_d84138ee1f_k_09.jpg
  Dibuat: 48597919241_d84138ee1f_k_10.jpg
  Dibuat: 48597919241_d84138ee1f_k_11.jpg
  Dibuat: 48597919241_d84138ee1f_k_12.jpg
  Dibuat: gusi_4_13.jpg
  Dibuat: gusi_4_14.jpg
  Dibuat: gusi_4_15.jpg
  Dibuat: gusi_4_16.jpg
  Dibuat: gusi_3_17.jpg
  Dibuat: gusi_3_18.jpg
  Dibuat: gusi_3_19.jpg
  Dibuat: gusi_3_20.jpg
  Dibuat: gusi_5_21.jpg
  Dibuat: gusi_5_22.jpg
  Dibuat: gusi_5_23.jpg
  Dibuat: gusi_5_24.jpg
Selesai! Total foto sekarang: 30

=== Memproses Kelas 2 ===
Foto asli: 6
Foto dibutuhkan: 24
Augmentasi per foto: 4
Augmentasi tambahan: 0


/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: gusi_1p_01.jpg
  Dibuat: gusi_1p_02.jpg
  Dibuat: gusi_1p_03.jpg
  Dibuat: gusi_1p_04.jpg
  Dibuat: Figure-4-Typical-foot-and-mouth-disease-oral-lesions-ulceration-after-rupture-of_Q320_05.jpg
  Dibuat: Figure-4-Typical-foot-and-mouth-disease-oral-lesions-ulceration-after-rupture-of_Q320_06.jpg
  Dibuat: Figure-4-Typical-foot-and-mouth-disease-oral-lesions-ulceration-after-rupture-of_Q320_07.jpg
  Dibuat: Figure-4-Typical-foot-and-mouth-disease-oral-lesions-ulceration-after-rupture-of_Q320_08.jpg
  Dibuat: gusi_6_09.jpg
  Dibuat: gusi_6_10.jpg
  Dibuat: gusi_6_11.jpg
  Dibuat: gusi_6_12.jpg
  Dibuat: gusi_2_13.jpg
  Dibuat: gusi_2_14.jpg
  Dibuat: gusi_2_15.jpg
  Dibuat: gusi_2_16.jpg
  Dibuat: gusi_4p_17.jpg
  Dibuat: gusi_4p_18.jpg
  Dibuat: gusi_4p_19.jpg
  Dibuat: gusi_4p_20.jpg
  Dibuat: dental pad lesions FMD_21.jpg
  Dibuat: dental pad lesions FMD_22.jpg
  Dibuat: dental pad lesions FMD_23.jpg
  Dibuat: dental pad lesions FMD_24.jpg
Selesai! Total foto sekarang: 30

==

/tmp/ipython-input-2-176338.py:11: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),


  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_01.jpg
  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_02.jpg
  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_03.jpg
  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_04.jpg
  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_05.jpg
  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_06.jpg
  Dibuat: FMD-generalized-lesions-in-cattle-6-days-after-exposure-to-cattle-inoculated-with-FMDV-O_07.jpg
  Dibuat: 8386722892_03ec82fcfd_o_08.jpg
  Dibuat: 8386722892_03ec82fcfd_o_09.jpg
  Dibuat: 8386722892_03ec82fcfd_o_10.jpg
  Dibuat: 8386722892_03ec82fcfd_o_11.jpg
  Dibuat: 8386722892_03ec82fcfd_o_12.jpg
  Dibuat: 8386722892_03ec82fcfd_o_13.jpg
  Dibuat: 83